In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as mlt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import LabelEncoder


In [ ]:
np.random.seed(42)
n_samples = 177000
df = pd.read_excel('/content/IndiaDataset.xlsx')


In [ ]:
label_encoder_state = LabelEncoder()
label_encoder_city = LabelEncoder()
df['State_Encoded'] = label_encoder_state.fit_transform(df['State'])
df['City_Encoded'] = label_encoder_city.fit_transform(df['City'])

In [ ]:
def calculate_aqi(o3, co, so2, no2, pm10, pm25):
    o3_aqi = np.interp(o3, [0, 50, 100, 200], [0, 50, 100, 150])
    co_aqi = np.interp(co, [0, 5, 10, 15], [0, 50, 100, 150])
    so2_aqi = np.interp(so2, [0, 40, 80, 380], [0, 50, 100, 150])
    no2_aqi = np.interp(no2, [0, 40, 80, 180], [0, 50, 100, 150])
    pm10_aqi = np.interp(pm10, [0, 50, 100, 250], [0, 50, 100, 150])
    pm25_aqi = np.interp(pm25, [0, 30, 60, 120], [0, 50, 100, 150])
    return max(o3_aqi, co_aqi, so2_aqi, no2_aqi, pm10_aqi, pm25_aqi)

# Calculate AQI for each row, update pm2.5 column name
df['AQI'] = df.apply(lambda row: calculate_aqi(row['Ozone'], row['CO'], row['SO2'],
                                               row['NO2'], row['PM10'], row['pm2_5']), axis=1)

KeyError: 'pm2_5'

In [ ]:
# Features and target
X = df[['ozone', 'co', 'no2', 'so2', 'pm10', 'pm2_5']].values
y = df['AQI'].values

# Normalize the data
X_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()
X_scaled = X_scaler.fit_transform(X)
y_scaled = y_scaler.fit_transform(y.reshape(-1, 1))

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

# Reshape for LSTM (samples, timesteps, features)
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [ ]:
model = Sequential([
    LSTM(128, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(1)
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=256, validation_data=(X_test, y_test))


Epoch 1/20
554/554 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - loss: 0.1215 - mae: 0.1999 - val_loss: 1.4098e-04 - val_mae: 0.0022
Epoch 2/20
554/554 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 1.5199e-04 - mae: 0.0038 - val_loss: 1.1664e-04 - val_mae: 0.0012
Epoch 3/20
554/554 ━━━━━━━━━━━━━━━━━━━━ 12s 13ms/step - loss: 8.0111e-05 - mae: 0.0014 - val_loss: 8.6519e-05 - val_mae: 9.3187e-04
Epoch 4/20
554/554 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - loss: 8.6056e-05 - mae: 0.0012 - val_loss: 5.6127e-05 - val_mae: 5.7011e-04
Epoch 5/20
554/554 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - loss: 3.9724e-05 - mae: 8.2364e-04 - val_loss: 3.7019e-05 - val_mae: 7.7546e-04
Epoch 6/20
554/554 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - loss: 5.1309e-05 - mae: 0.0011 - val_loss: 2.5593e-05 - val_mae: 2.7068e-04
Epoch 7/20
554/554 ━━━━━━━━━━━━━━━━━━━━ 7s 12ms/step - loss: 2.9689e-05 - mae: 7.7749e-04 - val_loss: 1.7285e-05 - val_mae: 0.0013
Epoch 8/20
554/554 ━━━━━━━━━━━━━━━━━━━━ 5s 9ms/step - loss: 2.6443e-05 - mae: 7.8632e-04 - v

In [ ]:
def predict_aqi(inputs):
    inputs_scaled = scaler.transform([inputs])
    inputs_reshaped = inputs_scaled.reshape((1, 1, len(inputs)))
    predicted_aqi_scaled = model.predict(inputs_reshaped)
    return scaler.inverse_transform(predicted_aqi_scaled)[0][0]

In [ ]:
y_test_pred_scaled = model.predict(X_test)

# Inverse transform predictions and true values to original scale
y_test_pred = scaler.inverse_transform(y_test_pred_scaled)
y_test_actual = scaler.inverse_transform(y_test)

# Calculate Evaluation Metrics
mse = mean_squared_error(y_test_actual, y_test_pred)
mae = mean_absolute_error(y_test_actual, y_test_pred)
r2 = r2_score(y_test_actual, y_test_pred)

print("\nModel Performance Metrics:")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"R^2 Score: {r2}")

1107/1107 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step


In [ ]:
test_loss, test_mae = model.evaluate(X_test, y_test, verbose=1)
print(f"Test Loss (MSE): {test_loss}")
print(f"Test MAE: {test_mae}")

1107/1107 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 1.6487e-05 - mae: 5.9469e-04
Test Loss (MSE): 1.5490262740058824e-05
Test MAE: 0.0005864948034286499


In [ ]:
def get_user_input():
    print("Enter the following input values:")
    ozone = float(input("Ozone (ppb): "))
    co = float(input("CO (ppm): "))
    no2 = float(input("NO2 (ppb): "))
    so2 = float(input("SO2 (ppb): "))
    pm10 = float(input("PM10 (µg/m³): "))
    pm2_5 = float(input("PM2.5 (µg/m³): "))


    return [ozone, co, no2, so2, pm10, pm2_5]

In [ ]:
def predict_aqi_for_user_input():
    """
    Function to process user input and predict AQI.
    """
    try:
        # Get user input
        user_inputs = get_user_input()

        # Scale and reshape input data
        user_inputs_scaled = X_scaler.transform([user_inputs])  # Use X_scaler
        user_inputs_reshaped = user_inputs_scaled.reshape((1, 1, len(user_inputs)))

        # Predict AQI
        predicted_aqi_scaled = model.predict(user_inputs_reshaped)
        predicted_aqi = y_scaler.inverse_transform(predicted_aqi_scaled)[0][0]  # Use y_scaler for inverse transform

        print(f"\nPredicted AQI for the given inputs: {predicted_aqi:.2f}")

    except ValueError as e:
        print(f"Error: {e}")


  #calling the function
predict_aqi_for_user_input()


Enter the following input values:
Ozone (ppb): 12
CO (ppm): 901
NO2 (ppb): 17
SO2 (ppb): 7
PM10 (µg/m³): 136
PM2.5 (µg/m³): 71
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step

Predicted AQI for the given inputs: 214.43
